<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [87]:
import numpy as np

### Datos

In [88]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [89]:
vector=[]
i=0
for documento in corpus:
    for termino in documento.split():
        vector.append(termino)
    print(documento.split())

vector=set(vector)
print(f"Terminos no repetidos en el corpus: {vector}")

['que', 'dia', 'es', 'hoy']
['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes']
['martes', 'muchas', 'gracias']
Terminos no repetidos en el corpus: {'muchas', 'dia', 'gracias', 'es', 'que', 'de', 'el', 'hoy', 'martes'}


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [90]:
def oneHotEnconder (vector,documento):
    listOneHot=[]
    #recorre todos los terminos del vector
    for termino in vector:
        #busca el termino en el documento o texto
        if termino in documento:
            listOneHot.append(1)
        else:
            listOneHot.append(0)
    return listOneHot

In [91]:
listOnehot=[]
for i in corpus:
    listOnehot.append(oneHotEnconder(vector,i)) 

In [92]:
listOnehot=np.array(listOnehot)
listOnehot

array([[0, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 0, 0, 0, 0, 1]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [93]:
def TF (vector, documento):
  listFrecuency=[]
  for i in vector:
    counter=0
    #busca cada termino i en los j terminos del documento
    for j in documento:
      if i==j:
        counter=counter+1
    listFrecuency.append(counter)
  return listFrecuency

In [94]:
listTF=[]

for documento in corpus:
    listTF.append(TF(vector,documento.split()))

listTF

[[0, 1, 0, 1, 1, 0, 0, 1, 0],
 [0, 1, 0, 1, 0, 1, 1, 1, 2],
 [1, 0, 1, 0, 0, 0, 0, 0, 1]]

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [95]:

def IDF (corpus, vector):
  listCounter=[]
  for termino in vector:
    counter=0
    # busca si el termino del vector se encuentra dentro de cada documento del corpus
    for documento in corpus:
      #si el termino esta aumenta el contador
      if termino in documento.split():
        counter=counter+1
    #Se calcula el factor IDF    
    N=len(corpus)
    listCounter.append(np.log(N/counter))
  return listCounter

In [96]:
idf=IDF(corpus,vector)
idf

[1.0986122886681098,
 0.4054651081081644,
 1.0986122886681098,
 0.4054651081081644,
 1.0986122886681098,
 1.0986122886681098,
 1.0986122886681098,
 0.4054651081081644,
 0.4054651081081644]

In [101]:
#multiplico el TF calculado anteriormente y lo multiplico por el factor IDF
tf_idf=np.array(listTF)*idf
tf_idf

array([[0.        , 0.40546511, 0.        , 0.40546511, 1.09861229,
        0.        , 0.        , 0.40546511, 0.        ],
       [0.        , 0.40546511, 0.        , 0.40546511, 0.        ,
        1.09861229, 1.09861229, 0.40546511, 0.81093022],
       [1.09861229, 0.        , 1.09861229, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.40546511]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [98]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

In [99]:
from itertools import combinations

#creo un diccionario con el indice es key y el value es el valor vectorizado de la frase
frases={idx:n_vector for idx,n_vector in enumerate (tf_idf)}

# Generar todas las combinaciones de 2 elementos con cada key
combinaciones = list(combinations(frases, 2))

#Aplico la funcion de similitud coseno a cada combinación 
scoreCos=[]
for i in combinaciones:
    scoreCos.append(cosine_similarity(frases[i[0]],frases[i[1]]))

#creo un diccionario donde la key es la similitud coseno y el value es la combinación
#de cada frase
similitudes=dict(zip(scoreCos,combinaciones))

#ordeno las keys de mayor a menor
keys_ordenados = sorted(similitudes.keys(),reverse=True)

In [100]:
keys_ordenados

[0.20034190268098706, 0.10845711727883084, 0.0]